# MELTS
  
Versions of MELTS implemented are:  
- MELTS v. 1.0.2 ➞ (rhyolite-MELTS, Gualda et al., 2012)  
- MELTS v. 1.1.0 ➞ (rhyolite-MELTS + new CO<sub>2</sub>, works at the ternary minimum)  
- MELTS v. 1.2.0 ➞ (rhyolite-MELTS + new H<sub>2</sub>O + new CO<sub>2</sub>)  
- pMELTS v. 5.6.1
- MELTS + DEW (experimental)

## Initialize tools and packages that are required to execute this notebook.

In [1]:
from thermoengine import equilibrate
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Create a MELTS (1.2.0) + DEW instance.
Rhyolite-MELTS version 1.2.0 + DEW.

In [2]:
melts = equilibrate.MELTSmodel('OnlyDEW')
# melts = equilibrate.MELTSmodel('DEW')

/Users/ghiorso/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Optional: Generate some information about the implemented model.

In [3]:
oxides = melts.get_oxide_names()
phases = melts.get_phase_names()
print (oxides)
print (phases)

['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'NiO', 'CoO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'H2O', 'CO2']
['DEWFluid', 'Liquid']


## Rhyolite: Input initial composition of the system (liquid), in wt% or grams of oxides. 
Early Bishop Tuff average melt inlusion composition

In [4]:
feasible = melts.set_bulk_composition({'SiO2':  77.5, 
                                       'TiO2':   0.08,# 0.08 
                                       'Al2O3': 12.5, # 12.5
                                       'Fe2O3':  0.0, # 0.207,
                                       'Cr2O3':  0.0, 
                                       'FeO':    0.0, # 0.473, 
                                       'MnO':    0.0,
                                       'MgO':    0.0, # 0.03, 
                                       'NiO':    0.0, 
                                       'CoO':    0.0,
                                       'CaO':    0.0, # 0.43, 
                                       'Na2O':   3.98,# 3.98 
                                       'K2O':    4.88,# 4.88 
                                       'P2O5':   0.0, 
                                       'H2O':   10.0, # 5.5
                                       'CO2':    0.05
                                      })

## Optional: Suppress phases that are not required in the simulation.

In [5]:
b = melts.get_phase_inclusion_status()
melts.set_phase_inclusion_status({'Actinolite' : False, 'Aegirine' : False, 'Aenigmatite' : False, 
                                  'Akermanite' : False, 'Andalusite' : False, 'Anthophyllite' : False, 
                                  'Apatite' : False, 'Augite' : False, 'Biotite' : False, 
                                  'Chromite' : False, 'Coesite' : False, 'Corundum' : True, 
                                  'Cristobalite' : True, 'Cummingtonite' : False, 'Fayalite' : False, 
                                  'Forsterite' : False, 'Garnet' : False, 'Gehlenite' : False, 
                                  'Hematite' : False, 'Hornblende' : False, 'Ilmenite' : False, 
                                  'Ilmenite ss' : False, 'Kalsilite' : False, 'Kalsilite ss' : False, 
                                  'Kyanite' : False, 'Leucite' : False, 'Lime' : False, 
                                  'Liquid Alloy' : False, 'Magnetite' : False, 'Melilite' : False, 
                                  'Muscovite' : False, 'Nepheline' : False, 'Nepheline ss' : False, 
                                  'Olivine' : False, 'OrthoOxide' : False, 'Orthopyroxene' : False, 
                                  'Panunzite' : False, 'Periclase' : False, 'Perovskite' : False, 
                                  'Phlogopite' : False, 'Pigeonite' : False, 'Plagioclase' : True, 
                                  'Quartz' : True, 'Rutile' : False, 'Sanidine' : True, 
                                  'Sillimanite' : False, 'Solid Alloy' : False, 'Sphene' : False, 
                                  'Spinel' : False, 'Titanaugite' : False, 'Tridymite' : True, 
                                  'Whitlockite' : False})
#a = melts.get_phase_inclusion_status()
#for phase in b.keys():
#    if b[phase] != a[phase]:
#        print ("{0:<15s} Before: {1:<5s} After: {2:<5s}".format(phase, repr(b[phase]), repr(a[phase])))

## Compute the equilibrium state at some specified T (°C) and P (MPa).
Print status and summary of the calculation.

In [6]:
melts.set_debug_state(debugS=True, debugV=False)
output = melts.equilibrate_tp(750.0, 500.0, initialize=True) # 175.0
(status, t, p, xmlout) = output[0]
print (status, t, p)
melts.output_summary(xmlout)

success, Minimal energy computed. 750.0 500.0
T (°C)       750.00
P (MPa)      500.00
DEWFluid          0.2758 (g)  DEWFluid - Formula NYI
                                 
Liquid          108.7142 (g)  wt%:SiO2 71.28 TiO2  0.07 Al2O3 11.50 Fe2O3  0.00 Cr2O3  0.00 FeO  0.00 MnO  0.00 MgO  0.00
                                  NiO  0.00 CoO  0.00 CaO  0.00 Na2O  3.66 K2O  4.49 P2O5  0.00 H2O  8.99 CO2  0.00


In [7]:
activities = melts.get_properties_of_DEWFluid(property_name='activity')
K = activities['H+']*activities['OH-']/activities['Water']
print ('pH = ', -np.log10(activities['H+']), 'neutral = ', -np.log10(np.sqrt(K)))

pH =  15.01097443023703 neutral =  13.98993025343966


In [8]:
print('Fluid composition in mole fraction:')
print(melts.get_composition_of_phase(xmlout, phase_name='DEWFluid', mode='component'))

Fluid composition in mole fraction:
{'Water': 0.911830629479316, 'CO2,aq': 0.0811068650366679, 'NaOH,aq': 2.83609868210128e-06, 'HAlO2,aq': 1.523252714218e-08, 'SiO2,aq': 0.0070596443602777, 'KOH,aq': 9.79252899912e-09}


In [9]:
print('Liquid composition in mole fraction:')
print(melts.get_composition_of_phase(xmlout,phase_name='Liquid',mode='component'))

Liquid composition in mole fraction:
{'SiO2': 0.589140622331367, 'TiO2': 0.000525774834794772, 'Al2O3': 0.0371739652955593, 'Na2SiO3': 0.0337201007395625, 'KAlSiO4': 0.0544043097662583, 'H2O': 0.285013929381353, 'CO2': 2.12976511049639e-05}


Output species present in Fluid phase

In [10]:
species = melts.get_properties_of_DEWFluid(property_name='species', T=750.0, P=1750.0)

In [11]:
for key in species.keys():
    if species[key] > 0.0:
        print ("{0:<20s} {1:12.6e}".format(key, species[key]))

KOH,aq               1.455872e-09
OH-                  4.802793e-07
CO2,aq               8.136754e-02
SiO2,aq              7.220981e-04
Water                9.146812e-01
HCO3-                8.791903e-07
NaOH,aq              1.878909e-06
NaHSiO3,aq           4.087810e-10
NaCO3-               1.113533e-12
NaHCO3,aq            1.283996e-10
Na+                  9.657971e-07
AlO2-                1.148150e-08
H+                   2.613747e-10
Al+3                 1.171759e-40
HSiO3-               1.793381e-04
K+                   8.368236e-09
Si2O4,aq             3.090487e-03
HAlO2,aq             3.800153e-09
CO3-2                2.808997e-11


## MORB: Input initial composition of the system (liquid), in wt% or grams of oxides. 
Composition from the FAMOUS fracture zone

In [ ]:
feasible = melts.set_bulk_composition({'SiO2':  48.68, 
                                       'TiO2':   1.01, 
                                       'Al2O3': 17.64, 
                                       'Fe2O3':  0.89,
                                       'Cr2O3':  0.0,  # 0.045
                                       'FeO':    7.59, 
                                       'MnO':    0.0,
                                       'MgO':    9.10, 
                                       'NiO':    0.0, 
                                       'CoO':    0.0,
                                       'CaO':   12.45, 
                                       'Na2O':   2.65, 
                                       'K2O':    0.03, 
                                       'P2O5':   0.0,  # 0.08
                                       'H2O':   20.0,
                                       'CO2':    0.0   # 0.05
                                      })

In [ ]:
melts.set_debug_state(debugS=True, debugV=True)
output = melts.equilibrate_tp(1200.0, 500.0, initialize=True)
(status, t, p, xmlout) = output[0]
print (status, t, p)
melts.output_summary(xmlout)

In [ ]:
activities = melts.get_properties_of_DEWFluid(property_name='activity')
K = activities['H+']*activities['OH-']/activities['Water']
print ('pH = ', -np.log10(activities['H+']), 'neutral = ', -np.log10(np.sqrt(K)))

In [ ]:
print('Fluid composition in mole fraction:')
print(melts.get_composition_of_phase(xmlout, phase_name='DEWFluid', mode='component'))

In [ ]:
print('Liquid composition in mole fraction:')
print(melts.get_composition_of_phase(xmlout,phase_name='Liquid',mode='component'))

Output species present in Fluid phase

In [ ]:
species = melts.get_properties_of_DEWFluid(property_name='species', T=750.0, P=1750.0)

In [ ]:
for key in species.keys():
    if species[key] > 0.0:
        print ("{0:<20s} {1:12.6e}".format(key, species[key]))

## Output chemical affinities and potential compositions of undersaturated phases 

In [ ]:
dict = melts.get_dictionary_of_affinities(xmlout, sort=True)
for phase in dict:
    (affinity, formulae) = dict[phase]
    print ("{0:<20s} {1:10.2f} {2:<60s}".format(phase, affinity, formulae))